In [2]:
import pandas as pd

# 读取 Excel 文件
data = pd.read_excel(r"W:\材智科技\中铝\测试数据.xlsx")

In [3]:
data.iloc[:-10,:4].to_numpy()

array([[0.000e+00, 4.741e+00, 3.050e+02, 5.200e+02],
       [3.700e-02, 4.807e+00, 3.050e+02, 5.200e+02],
       [3.700e-02, 4.943e+00, 3.050e+02, 5.200e+02],
       ...,
       [3.600e-01, 5.700e+00, 2.850e+02, 4.800e+02],
       [3.600e-01, 5.700e+00, 2.850e+02, 4.800e+02],
       [3.600e-01, 5.700e+00, 2.850e+02, 5.000e+02]])

In [4]:
data.iloc[:-10,4:].to_numpy()

array([[117],
       [120],
       [128],
       [116],
       [116],
       [117],
       [117],
       [115],
       [113],
       [118],
       [129],
       [126],
       [138],
       [128],
       [120],
       [121],
       [127],
       [129],
       [121],
       [124],
       [128],
       [127],
       [122],
       [117],
       [108],
       [123],
       [116],
       [130],
       [123],
       [125],
       [105],
       [120],
       [135],
       [122],
       [121],
       [112],
       [120],
       [122],
       [128],
       [122],
       [122],
       [126],
       [121],
       [122],
       [135],
       [122],
       [124],
       [125],
       [122],
       [118],
       [134],
       [118],
       [121],
       [110],
       [124],
       [121],
       [127],
       [128],
       [128],
       [129],
       [126],
       [134],
       [135],
       [123],
       [116],
       [118],
       [130],
       [134],
       [111],
       [101],
       [113],
      

In [12]:
import numpy as np

alpha = 0.000001
hidden_size_1 = 128
hidden_size_2 = 128
hidden_size_3 = 128
epoch = 100000

layer_0_1_weights = 2 * np.random.random((4, hidden_size_1)) - 1
layer_1_2_weights = 2 * np.random.random((hidden_size_1, hidden_size_2)) - 1
layer_2_3_weights = 2 * np.random.random((hidden_size_2, hidden_size_3)) - 1
layer_3_4_weights = 2 * np.random.random((hidden_size_3, 1)) - 1

def relu(x):
    return (x > 0) * x


def relu_origin(x):
    return x > 0

# 计算R²值的函数
def calculate_r2(y_true, y_pred):
    """
    计算R²（决定系数）
    R² = 1 - (SSE / SST)
    其中：
    SSE = Sum of Squared Errors = Σ(y_true - y_pred)²
    SST = Total Sum of Squares = Σ(y_true - y_mean)²
    """
    # 确保输入是numpy数组
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()

    # 计算真实值的均值
    y_mean = np.mean(y_true)

    # 计算SSE（残差平方和）
    sse = np.sum((y_true - y_pred) ** 2)

    # 计算SST（总平方和）
    sst = np.sum((y_true - y_mean) ** 2)

    # 计算R²
    if sst == 0:
        return 1.0 if sse == 0 else 0.0

    r2 = 1 - (sse / sst)
    return r2

# street_lights = np.array([
#     [0, 4.741, 305, 520],
#     [0.037, 4.807, 305, 520],
#     [0.037, 4.943, 305, 520],
#     [0.037, 4.943, 305, 520],
#     [0.037, 4.807, 305, 520],
#     [0.006, 4.918, 305, 520],
#     [0.037, 4.943, 305, 520],
#     [0.037, 4.807, 305, 520],
#     [0.041, 3.48, 320, 500],
#     [0.022, 4.87, 305, 520]
# ])
street_lights = data.iloc[:-10,:4].to_numpy()

# 计算每列的均值和标准差
inputs_mean = np.mean(street_lights, axis=0)
inputs_std = np.std(street_lights, axis=0)

# 标准化
street_lights = (street_lights - inputs_mean) / inputs_std
# street_lights = street_lights/100

# walk_stop = np.array([
#     [117],
#     [120],
#     [128],
#     [116],
#     [116],
#     [117],
#     [117],
#     [115],
#     [113],
#     [118]
# ])
walk_stop = data.iloc[:-10,4:].to_numpy()

# 计算每列的均值和标准差
outputs_mean = np.mean(walk_stop, axis=0)
outputs_std = np.std(walk_stop, axis=0)

# 标准化
walk_stop = (walk_stop - outputs_mean) / outputs_std
# walk_stop = walk_stop/100

for n in range(epoch):
    all_error = 0
    predictions = []
    for i in range(len(street_lights)):
        inputs = street_lights[i]
        layer_1 = relu(inputs.dot(layer_0_1_weights))
        layer_2 = relu(layer_1.dot(layer_1_2_weights))
        layer_3 = relu(layer_2.dot(layer_2_3_weights))
        outputs = layer_3.dot(layer_3_4_weights)

        error = np.sum((outputs - walk_stop[i]) ** 2)

        layer_4_delta = outputs - walk_stop[i]
        layer_3_delta = layer_4_delta.dot(layer_3_4_weights.T) * relu_origin(layer_3)
        layer_2_delta = layer_3_delta.dot(layer_2_3_weights.T) * relu_origin(layer_2)
        layer_1_delta = layer_2_delta.dot(layer_1_2_weights.T) * relu_origin(layer_1)

        layer_3_4_delta_weight = layer_3.reshape(hidden_size_3, 1).dot(layer_4_delta.reshape(1, 1))  # (10, 1)
        layer_2_3_delta_weight = layer_2.reshape(hidden_size_2, 1).dot(layer_3_delta.reshape(1, hidden_size_3))  # (10, 10)
        layer_1_2_delta_weight = layer_1.reshape(hidden_size_1, 1).dot(layer_2_delta.reshape(1, hidden_size_2))  # (10, 10)
        layer_0_1_delta_weight = inputs.reshape(4, 1).dot(layer_1_delta.reshape(1, hidden_size_1))  # (4, 10)

        layer_3_4_weights -= alpha * layer_3_4_delta_weight
        layer_2_3_weights -= alpha * layer_2_3_delta_weight
        layer_1_2_weights -= alpha * layer_1_2_delta_weight
        layer_0_1_weights -= alpha * layer_0_1_delta_weight
        all_error += error
        predictions.append(outputs[0])
        if (n % 100 == 0 and i < 10) or (n == epoch - 1 and i < 10):
            pass
            #print('Predict:', outputs * outputs_std + outputs_mean)
            # print('Predict:', layer_3 * 100)
    if n % 100 == 0 or n == epoch - 1:
        print('R²:', calculate_r2(walk_stop, predictions))
        print('Error:', all_error)

R²: -1329.9738998771277
Error: 537713.4555503597
R²: 0.006556236918113689
Error: 401.35128028508194
R²: 0.3698134110608995
Error: 254.59538193139673
R²: 0.4804428163229877
Error: 209.90110220551264
R²: 0.5370491110360128
Error: 187.03215914145076
R²: 0.5696086413366312
Error: 173.87810890000097
R²: 0.591607464501652
Error: 164.99058434133252
R²: 0.6069723087345205
Error: 158.7831872712539
R²: 0.6177879456476134
Error: 154.413669958364
R²: 0.6267646955822255
Error: 150.78706298478093
R²: 0.6340853133757559
Error: 147.82953339619468
R²: 0.6402846196643437
Error: 145.32501365560518
R²: 0.6447665823107366
Error: 143.5143007464623
R²: 0.6481691276470223
Error: 142.13967243060316
R²: 0.651347015487212
Error: 140.85580574316634
R²: 0.6541475796832281
Error: 139.72437780797586
R²: 0.6566333161092193
Error: 138.72014029187545
R²: 0.6589745884219951
Error: 137.7742662775141
R²: 0.661669194185367
Error: 136.6856455491117
R²: 0.6638286584670485
Error: 135.81322197931246
R²: 0.6653554364076965
Erro

KeyboardInterrupt: 

R²: 0.6781498706272087

Error: 130.02745226660758

R²: 0.6784385007003134

Error: 129.91084571707336

R²: 0.6789350866300339

Error: 129.71022500146643

R²: 0.6789188822773411

Error: 129.7167715599543


In [16]:
a = np.array([[1], [0], [1]])
b = np.array([[1.1, 2.1, 3.1, 4.1]])

print('a:', a)
print('b:', b)
print(a.dot(b))

a: [[1]
 [0]
 [1]]
b: [[1.1 2.1 3.1 4.1]]
[[1.1 2.1 3.1 4.1]
 [0.  0.  0.  0. ]
 [1.1 2.1 3.1 4.1]]


In [13]:
len(street_lights)

10